# Face Detection Backend Comparison

This notebook compares different face detection backends available in FaceVerify.

## Table of Contents
1. Available Backends
2. Detection Speed Comparison
3. Detection Accuracy Comparison
4. Visualization
5. Recommendations

In [ ]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from faceverify import FaceVerifier
from faceverify.config import VerifierConfig
from faceverify.detection import FaceDetector

## 1. Available Backends

FaceVerify supports multiple face detection backends:

In [ ]:
# List available detection backends
BACKENDS = [
    "opencv",      # OpenCV Haar Cascade (fastest, least accurate)
    "mtcnn",       # Multi-task CNN (balanced)
    "retinaface", # RetinaFace (most accurate, slower)
    "mediapipe"    # MediaPipe (fast, good accuracy)
]

print("Available Face Detection Backends")
print("=" * 50)
for i, backend in enumerate(BACKENDS, 1):
    print(f"{i}. {backend}")

In [ ]:
# Check which backends are actually available
detector = FaceDetector()
available = detector.get_available_backends()

print("\nInstalled backends on this system:")
for backend in available:
    print(f"  [OK] {backend}")

## 2. Detection Speed Comparison

Measure detection speed for each backend:

In [ ]:
# Test image path
TEST_IMAGE = "../test_images/person1_a.jpg"

# Load image once
image = cv2.imread(TEST_IMAGE)
if image is None:
    print(f"Error: Could not load {TEST_IMAGE}")
    print("Please update TEST_IMAGE path to a valid image.")
else:
    print(f"Loaded image: {image.shape}")

In [ ]:
def benchmark_backend(backend_name, image_path, num_runs=5):
    """Benchmark a detection backend."""
    try:
        config = VerifierConfig(detector_backend=backend_name)
        verifier = FaceVerifier(config=config)
        
        times = []
        for _ in range(num_runs):
            start = time.time()
            faces = verifier.detect_faces(image_path)
            elapsed = time.time() - start
            times.append(elapsed)
        
        avg_time = np.mean(times)
        std_time = np.std(times)
        num_faces = len(faces) if faces else 0
        
        return {
            "backend": backend_name,
            "avg_time": avg_time,
            "std_time": std_time,
            "faces_detected": num_faces,
            "status": "OK"
        }
    except Exception as e:
        return {
            "backend": backend_name,
            "avg_time": 0,
            "std_time": 0,
            "faces_detected": 0,
            "status": f"Error: {str(e)[:50]}"
        }

In [ ]:
# Run benchmarks
print("Running benchmarks (this may take a minute)...")
print()

results = []
for backend in BACKENDS:
    print(f"Testing {backend}...", end=" ")
    result = benchmark_backend(backend, TEST_IMAGE)
    results.append(result)
    if result["status"] == "OK":
        print(f"Done ({result['avg_time']:.3f}s)")
    else:
        print(result["status"])

In [ ]:
# Display results table
print("\nBenchmark Results")
print("=" * 70)
print(f"{'Backend':<15} {'Avg Time':<12} {'Std Dev':<12} {'Faces':<8} {'Status'}")
print("-" * 70)

for r in results:
    if r["status"] == "OK":
        print(f"{r['backend']:<15} {r['avg_time']:.4f}s      {r['std_time']:.4f}s      {r['faces_detected']:<8} {r['status']}")
    else:
        print(f"{r['backend']:<15} {'N/A':<12} {'N/A':<12} {'N/A':<8} {r['status']}")

## 3. Detection Accuracy Comparison

Compare bounding box accuracy across backends:

In [ ]:
def get_detection_details(backend_name, image_path):
    """Get detailed detection information."""
    try:
        config = VerifierConfig(detector_backend=backend_name)
        verifier = FaceVerifier(config=config)
        faces = verifier.detect_faces(image_path)
        return faces
    except Exception as e:
        print(f"Error with {backend_name}: {e}")
        return []

In [ ]:
# Get detections from each backend
all_detections = {}
for backend in BACKENDS:
    faces = get_detection_details(backend, TEST_IMAGE)
    all_detections[backend] = faces
    print(f"{backend}: {len(faces)} face(s) detected")

## 4. Visualization

Visualize detection results:

In [ ]:
def draw_detections(image, faces, backend_name, color):
    """Draw bounding boxes on image."""
    img_copy = image.copy()
    
    for face in faces:
        if hasattr(face, 'bbox'):
            x, y, w, h = face.bbox
        elif isinstance(face, dict) and 'bbox' in face:
            x, y, w, h = face['bbox']
        else:
            continue
            
        cv2.rectangle(img_copy, (int(x), int(y)), (int(x+w), int(y+h)), color, 2)
        cv2.putText(img_copy, backend_name, (int(x), int(y)-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    return img_copy

In [ ]:
# Create comparison visualization
if image is not None:
    colors = [
        (255, 0, 0),    # Blue for opencv
        (0, 255, 0),    # Green for mtcnn
        (0, 0, 255),    # Red for retinaface
        (255, 255, 0)   # Cyan for mediapipe
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for idx, (backend, color) in enumerate(zip(BACKENDS, colors)):
        faces = all_detections.get(backend, [])
        img_with_boxes = draw_detections(image, faces, backend, color)
        img_rgb = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(img_rgb)
        axes[idx].set_title(f"{backend} ({len(faces)} faces)")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig("detection_comparison.png", dpi=150)
    plt.show()
    print("Saved comparison to detection_comparison.png")

## 5. Recommendations

Based on the benchmarks, here are recommendations for different use cases:

In [ ]:
recommendations = """
BACKEND RECOMMENDATIONS
=======================

1. REAL-TIME APPLICATIONS (webcam, video)
   Recommended: opencv or mediapipe
   Reason: Fastest detection speed

2. HIGH ACCURACY REQUIRED (security, verification)
   Recommended: retinaface
   Reason: Most accurate bounding boxes and landmarks

3. BALANCED (general purpose)
   Recommended: mtcnn
   Reason: Good balance of speed and accuracy

4. RESOURCE CONSTRAINED (embedded, mobile)
   Recommended: opencv
   Reason: Lowest memory and CPU usage

5. MULTIPLE FACES IN IMAGE
   Recommended: mtcnn or retinaface
   Reason: Better at detecting multiple faces
"""

print(recommendations)

In [ ]:
# Summary table
summary = """
SUMMARY TABLE
=============

| Backend    | Speed    | Accuracy | Multi-face | GPU Support |
|------------|----------|----------|------------|-------------|
| opencv     | Fast     | Low      | Fair       | No          |
| mtcnn      | Medium   | High     | Good       | Yes         |
| retinaface | Slow     | Highest  | Excellent  | Yes         |
| mediapipe  | Fast     | Medium   | Good       | Yes         |
"""

print(summary)

## Next Steps

Continue to `03_embedding_analysis.ipynb` to learn about face embedding models.